# Scopo del notebook di Deploy

Lo scopo di questa parte è quella di fornire un notebook/script in grado di applicare il modello su nuovi dati e riportare i corrispondenti risultati di classificazione, regressione o clustering. Si ipotizzi che i nuovi dati siano forniti con lo stesso formato e struttura dei dati originali.

# Settings

In [ ]:
# IMPORTAZIONE LIBRERIE
import cv2
import pickle
import random
import glob

# PACCHETTI DI BASE
import os
import numpy as np
import pandas as pd

# SCIKIT-IMAGE
from skimage.io import imread

In [ ]:
DIR_INFERENCE_DATA = '../Data/Inference/DATA/'
NUM_IMG = 20
SCALE_PERCENT = 6
random.seed(123)

# Parametri di cropping: creazione di una finestra per il ritagliamento dell'immagine
x=800
w=3000
y=2000
h=4200

# Scaler Loading

In [ ]:
with open('../Models/minMax_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

scaler

# Inference dataset Creation

In [ ]:
# Selezione di 29 immagini prese in modo casuale dalla cartella contenente le immagini originali

file_path_type = ["../Data/Original/OK/*.bmp"]
images = glob.glob(random.choice(file_path_type))
for i in range(NUM_IMG):
    random_image = random.choice(images)
    print(random_image)
    img_raw = imread(random_image)
    cv2.imwrite(os.path.join(DIR_INFERENCE_DATA , os.path.basename(random_image)), img_raw)


file_path_type = [ "../Data/Original/NOK/*.bmp"]
images = glob.glob(random.choice(file_path_type))
for i in range(NUM_IMG):
    random_image = random.choice(images)
    print(random_image)
    img_raw = imread(random_image)
    cv2.imwrite(os.path.join(DIR_INFERENCE_DATA , os.path.basename(random_image)), img_raw)

In [ ]:
# Viene eseguita una ridenominazione delle immagini selezionate in modo casuale in modo tale da riprodurre con più accuratezza possibile uno scenario reale

count = 1

for file_name in os.listdir(DIR_INFERENCE_DATA):
    source = DIR_INFERENCE_DATA + file_name
    destination = DIR_INFERENCE_DATA + "IMG_" + str(count) + ".bmp"
    os.rename(source, destination)
    count += 1
print('All Files Renamed')

print('New Names are')
res = os.listdir(DIR_INFERENCE_DATA)
print(res)

# Dataset preparation

In [ ]:
image_names = []  # struttura dati contenente i nomi delle immagini
VECTOR_DATA = []  # struttura dati contenente le immagini in formato vettoriale

for img in os.listdir(DIR_INFERENCE_DATA):
    print('sto processando una immagine...')

    image_names.append(str(img))

    # Selezione immagine singola
    img = imread(os.path.join(DIR_INFERENCE_DATA,img))
    # Conversione in bianco e nero
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Cropping
    img_crop = img_gray[x:w, y:h]
    # Ridimensionamento
    width1 = int(img_crop.shape[1] * SCALE_PERCENT / 100)
    height1 = int(img_crop.shape[0] * SCALE_PERCENT / 100)
    dim1 = (width1, height1)
    img_resized = cv2.resize(img_crop, dim1, interpolation = cv2.INTER_AREA)
    # Conversione in forma vettoriale
    VECTOR_DATA.append(img_resized.flatten())

VECTOR_DATA = np.array(VECTOR_DATA)
X = pd.DataFrame(VECTOR_DATA)
X

In [ ]:
# Applicazione dello scaler sui dati

X = pd.DataFrame(
    scaler.transform(X),
    index=X.index
)
X

# Model loading

In [ ]:
with open('../Models/support_vector_machine.pkl', 'rb') as file:
    model = pickle.load(file)
model

# Prediction

In [ ]:
# Predizioni sui valori di inference

y_pred = pd.Series(model.predict(X), name='prediction')
y_pred = pd.DataFrame(y_pred)
y_pred['name'] = image_names
y_pred

# Prediction persistence

In [ ]:
y_pred.to_csv('../Data/Inference/bosch_predictions.csv', index=False)